### Load IMDb dataset

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset('imdb')

### Tokenize data

In [ ]:
from transformers import AutoTokenizer

model_name = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Alternatively:
# from transformers import BertTokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def tokenize(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

tokenized_datasets = raw_datasets.map(tokenize, batched=True)

### Make small datasets for faster training

In [ ]:
small_train_dataset = tokenized_datasets['train'].shuffle(seed=7).select(range(1000))
small_eval_dataset = tokenized_datasets['test'].shuffle(seed=7).select(range(1000))
full_train_dataset = tokenized_datasets['train']
full_eval_dataset = tokenized_datasets['test']

### Define model

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Alternatively:
# from transformers import BertForSequenceClassification
# model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

### Fine-tune model

In [ ]:
import numpy as np
from datasets import load_metric
from transformers import Trainer, TrainingArguments

data_size = 'small'  # small or full

accuracy_metric = load_metric('accuracy')
precision_metric = load_metric('precision')
recall_metric = load_metric('recall')
f1_metric = load_metric('f1')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    accuracy = accuracy_metric.compute(references=labels, predictions=predictions)
    precision = precision_metric.compute(references=labels, predictions=predictions)
    recall = recall_metric.compute(references=labels, predictions=predictions)
    f1 = f1_metric.compute(references=labels, predictions=predictions)
    
    return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}

output_dir = 'test_model'
training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy='steps',
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset if data_size == 'small' else full_train_dataset,
    eval_dataset=small_eval_dataset if data_size == 'small' else full_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()